# 1. 라이브러리 불러오기

In [1]:
import pandas as pd
import os
import dask.dataframe as dd
import datetime
import calendar
import glob

# 2. 사용자 설정

2-1. default 경로 입력

In [2]:
default_dir = 'C:/19Project/05.TRV/02.QGIS_workspace/'
pop_data_dir = default_dir + 'input/유동인구'

2-2. 사용 기간, 시간 설정

In [3]:
##### 사용 기간 설정 #####
start_ym = 201907
end_ym = 201908

##### 사용 시간 설정 #####
start_h = 7
end_h = 22

# 3. 데이터 불러오기

3-1. 디렉터리 생성

In [4]:
now = datetime.datetime.now().date().strftime('%Y%m%d')

num = 1

preprocessing_dir = default_dir + '유동인구 전처리_' + now

while os.path.exists(preprocessing_dir + '_' + str(num)):
    num += 1
    
preprocessing_dir_num = preprocessing_dir + '_' + str(num) + '/'

os.makedirs(os.path.join(preprocessing_dir_num))

3-2. 유동인구 데이터 불러오기

In [5]:
all_files = glob.glob(pop_data_dir + "/*.csv")

df = dd.read_csv(all_files, engine = 'python', encoding = 'UTF-8', sep = '|', skiprows = [0], header = None)
raw_pop_data = df.compute()
raw_pop_data.columns = ['연월', 'gid', 'ADM_DR_CD', '시각', 'FLOW_POP_CNT_6000D', 'FLOW_POP_CNT_6064', 'FLOW_POP_CNT_6569', 
                        'FLOW_POP_CNT_7074', 'FLOW_POP_CNT_7579', 'FLOW_POP_CNT_8084', 'FLOW_POP_CNT_8589', 'FLOW_POP_CNT_9000U']

# 4. 함수 생성

In [6]:
# grid, 시각별 집계 - 유동인구
def shuttle_pop(pop_data):
    # 연월, 시각 필터링
    pop_data_ym = pop_data[(pop_data['연월'] >= start_ym) & (pop_data['연월'] <= end_ym)]
    pop_data_ym = pop_data_ym[(pop_data_ym['시각'] >= start_h) & (pop_data_ym['시각'] <= end_h)]
    pop_data_ym.reset_index(drop = True, inplace = True)
    # 유동인구 sum 계산
    pop_data_ym['date_ym'] = pd.to_datetime(pop_data_ym['연월'], format = '%Y%m')
    pop_data_ym['date_d'] = pop_data_ym['date_ym'].apply(lambda x: calendar.monthrange(x.year, x.month)[1])
    pop_data_ym['pop_6569'] = pop_data_ym['date_d'] * pop_data_ym['FLOW_POP_CNT_6569']
    pop_data_ym['pop_7074'] = pop_data_ym['date_d'] * pop_data_ym['FLOW_POP_CNT_7074']
    pop_data_ym['pop_7579'] = pop_data_ym['date_d'] * pop_data_ym['FLOW_POP_CNT_7579']
    pop_data_ym['pop_8084'] = pop_data_ym['date_d'] * pop_data_ym['FLOW_POP_CNT_8084']
    pop_data_ym['pop_8589'] = pop_data_ym['date_d'] * pop_data_ym['FLOW_POP_CNT_8589']
    pop_data_ym['pop_90up'] = pop_data_ym['date_d'] * pop_data_ym['FLOW_POP_CNT_9000U']
    pop_data_ym['유동인구수'] = pop_data_ym['pop_6569'] + pop_data_ym['pop_7074'] + pop_data_ym['pop_7579'] + pop_data_ym['pop_8084'] + pop_data_ym['pop_8589'] + pop_data_ym['pop_90up']
    pop_data_ym = pop_data_ym[['연월', 'gid', '시각', '유동인구수']]
    # grid, 시각별 집계
    pop_table = pd.DataFrame(pop_data_ym.groupby(['gid', '시각'])['유동인구수'].sum())
    pop_table.reset_index(inplace = True)
    pop_table.columns = ['gid', '시각', '유동인구수']
    # 유동인구 sum값이 0인 경우 제외
    pop_table = pop_table[pop_table['유동인구수'] > 0]
    pop_table.to_csv(preprocessing_dir_num + 'grid별 시각별 유동인구 index_' + str(start_ym) + "_" + str(end_ym) + '.csv',
                    index = False, encoding = 'MS949')
    print('[유동인구 index 저장] \n' + preprocessing_dir_num + 'grid별 시각별 유동인구 index_' + str(start_ym) + "_" + str(end_ym) + '.csv')

# 5. 유동인구 데이터 전처리

In [7]:
shuttle_pop(raw_pop_data)

[유동인구 index 저장] 
C:/19Project/05.TRV/02.QGIS_workspace/유동인구 전처리_20211012_1/grid별 시각별 유동인구 index_201907_201908.csv
